# S-Bert caption representation and Pairwise Cosine Similarity

In [1]:
import json
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
import torch

In [2]:
bert = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

## COCO

In [3]:
f1 = '/data/project/rw/woong.ssang/graph_matching/txt2img_new/data/coco/annotations/captions_train2014.json'
cocotr = json.load(open(f1, 'r'))
f2 = '/data/project/rw/woong.ssang/graph_matching/txt2img_new/data/coco/annotations/captions_val2014.json'
cocoval = json.load(open(f2, 'r'))

In [4]:
l_sent_id = [t['id'] for t in cocotr['annotations']] + [t['id'] for t in cocoval['annotations']]

In [34]:
l_img_id = [t['image_id'] for t in cocotr['annotations']] + [t['image_id'] for t in cocoval['annotations']]


In [6]:
d_img_id2idx = {img_id: idx for idx, img_id in enumerate(sorted(list(set(l_img_id))))}

In [5]:
l_sent = [t['caption'] for t in cocotr['annotations']] + [t['caption'] for t in cocoval['annotations']]

In [7]:
l_sent[:3]

['A very clean and well decorated empty bathroom',
 'A panoramic view of a kitchen and all of its appliances.',
 'A blue and white bathroom with butterfly themed wall tiles.']

In [8]:
l_emb = bert.encode(l_sent, batch_size=128, show_progress_bar=True, )

Batches: 100%|██████████| 4819/4819 [14:57<00:00,  5.37it/s]


In [12]:
l_emb = np.array(l_emb)

In [17]:
l_norm_emb = l_emb / np.sqrt((l_emb ** 2).sum(axis=1, keepdims=True))

In [14]:
np.save('coco_sbert_emb.npy', {'image_id':np.array(l_img_id), 
                               'sent_id': np.array(l_sent_id),
                               'normed_sbert_emb': l_norm_emb})

In [2]:
d = np.load('coco_sbert_emb.npy', allow_pickle=True)

In [3]:
d = d.tolist()
l_img_id = d['imgage_id']
l_sent_id = d['sent_id']
l_norm_emb = d['normed_sbert_emb']

In [4]:
B = np.zeros((123287, 5, 1024), dtype='float32')
cap_count = np.zeros((123287,), dtype='int')

In [7]:
for i, (sent_id, img_id, emb) in enumerate(zip(l_sent_id, l_img_id, l_norm_emb)):
    img_idx = d_img_id2idx[img_id]
    sent_idx = cap_count[img_idx]
    if sent_idx >= 5:
        # skip more captions
#         print(f'{img_id}')
        continue
    B[img_idx, sent_idx, :] = emb
    cap_count[img_idx] += 1

In [8]:
n_img = len(B)
n_batch = 20
batch_size = int(n_img / n_batch)
pbar = tqdm(total=n_batch ** 2)
l_row = []
for i in range(n_batch):
    i_s = i * batch_size
    if i == n_batch - 1:
        i_e = n_img
    else:
        i_e = (i + 1) * batch_size
        
    B_i = B[i_s:i_e]
    B_i = torch.from_numpy(B_i).cuda()
    
    l_B = []
    for j in range(n_batch):
#         print(j)
        j_s = j * batch_size
        if j == n_batch - 1:
            j_e = n_img
        else:
            j_e = (j + 1) * batch_size
            
        B_j = B[j_s:j_e]
        B_j = torch.from_numpy(B_j).cuda()
        
        BB = torch.einsum('ijk,lmk->iljm', B_i, B_j)
        BB = BB.reshape((len(B_i), len(B_j), -1)).mean(axis=-1)
        
        l_B.append(BB.cpu())
        
        pbar.update(1)
        
    row = np.hstack(l_B)
    l_row.append(row)

M = np.vstack(l_row)

100%|██████████| 400/400 [03:04<00:00,  3.17it/s]

In [9]:
np.save('coco_sbert_mean.npy', M)

In [13]:
M.shape

(123287, 123287)

In [15]:
np.save('coco_sbert_img_id.npy', sorted(list(set(l_img_id))))

------

## Flickr 30K

In [3]:
f = '/data/project/rw/woong.ssang/graph_matching/txt2img_new/data/f30k/dataset_flickr30k.json'
f30k = json.load(open(f, 'r'))

In [6]:
f30k['images'][0]['sentences'][0]['raw']

'Two young guys with shaggy hair look at their hands while hanging out in the yard.'

In [8]:
f30k['images'][0]

{'sentids': [0, 1, 2, 3, 4],
 'imgid': 0,
 'sentences': [{'tokens': ['two',
    'young',
    'guys',
    'with',
    'shaggy',
    'hair',
    'look',
    'at',
    'their',
    'hands',
    'while',
    'hanging',
    'out',
    'in',
    'the',
    'yard'],
   'raw': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
   'imgid': 0,
   'sentid': 0},
  {'tokens': ['two',
    'young',
    'white',
    'males',
    'are',
    'outside',
    'near',
    'many',
    'bushes'],
   'raw': 'Two young, White males are outside near many bushes.',
   'imgid': 0,
   'sentid': 1},
  {'tokens': ['two',
    'men',
    'in',
    'green',
    'shirts',
    'are',
    'standing',
    'in',
    'a',
    'yard'],
   'raw': 'Two men in green shirts are standing in a yard.',
   'imgid': 0,
   'sentid': 2},
  {'tokens': ['a',
    'man',
    'in',
    'a',
    'blue',
    'shirt',
    'standing',
    'in',
    'a',
    'garden'],
   'raw': 'A man in a blue shirt standing in

In [25]:
l_sent_id = [s['sentid'] for t in f30k['images'] for s in t['sentences']] 

In [27]:
l_img_id = [s['imgid'] for t in f30k['images'] for s in t['sentences']]

In [28]:
d_img_id2idx = {img_id: idx for idx, img_id in enumerate(sorted(list(set(l_img_id))))}

In [29]:
l_sent = [s['raw'] for t in f30k['images'] for s in t['sentences']]

In [30]:
l_sent[:3]

['Two young guys with shaggy hair look at their hands while hanging out in the yard.',
 'Two young, White males are outside near many bushes.',
 'Two men in green shirts are standing in a yard.']

In [33]:
l_emb = bert.encode(l_sent, batch_size=128, show_progress_bar=True, )


Batches: 100%|██████████| 1212/1212 [04:16<00:00,  4.73it/s]


In [34]:
l_emb = np.array(l_emb)

In [35]:
l_norm_emb = l_emb / np.sqrt((l_emb ** 2).sum(axis=1, keepdims=True))

In [36]:
np.save('f30k_sbert_emb.npy', {'image_id':np.array(l_img_id), 
                               'sent_id': np.array(l_sent_id),
                               'normed_sbert_emb': l_norm_emb})

In [37]:
len(l_norm_emb)

155070

In [42]:
from collections import Counter
max([j for i,j in Counter(l_img_id).items()])

5

In [46]:
n_img = len(list(set(l_img_id)))

In [2]:
# d = np.load('coco_sbert_emb.npy', allow_pickle=True)

In [3]:
# d = d.tolist()
# l_img_id = d['imgage_id']
# l_sent_id = d['sent_id']
# l_norm_emb = d['normed_sbert_emb']

In [47]:
B = np.zeros((n_img, 5, 1024), dtype='float32')
cap_count = np.zeros((n_img,), dtype='int')

In [48]:
for i, (sent_id, img_id, emb) in enumerate(zip(l_sent_id, l_img_id, l_norm_emb)):
    img_idx = d_img_id2idx[img_id]
    sent_idx = cap_count[img_idx]
    if sent_idx >= 5:
        # skip more captions
#         print(f'{img_id}')
        continue
    B[img_idx, sent_idx, :] = emb
    cap_count[img_idx] += 1

In [49]:
n_img = len(B)
n_batch = 20
batch_size = int(n_img / n_batch)
pbar = tqdm(total=n_batch ** 2)
l_row = []
for i in range(n_batch):
    i_s = i * batch_size
    if i == n_batch - 1:
        i_e = n_img
    else:
        i_e = (i + 1) * batch_size
        
    B_i = B[i_s:i_e]
    B_i = torch.from_numpy(B_i).cuda()
    
    l_B = []
    for j in range(n_batch):
#         print(j)
        j_s = j * batch_size
        if j == n_batch - 1:
            j_e = n_img
        else:
            j_e = (j + 1) * batch_size
            
        B_j = B[j_s:j_e]
        B_j = torch.from_numpy(B_j).cuda()
        
        BB = torch.einsum('ijk,lmk->iljm', B_i, B_j)
        BB = BB.reshape((len(B_i), len(B_j), -1)).mean(axis=-1)
        
        l_B.append(BB.cpu())
        
        pbar.update(1)
        
    row = np.hstack(l_B)
    l_row.append(row)

M = np.vstack(l_row)


100%|██████████| 400/400 [00:28<00:00, 39.02it/s]

In [50]:
np.save('f30k_sbert_mean.npy', M)

In [51]:
M.shape

(31014, 31014)

In [52]:
np.save('f30k_sbert_img_id.npy', sorted(list(set(l_img_id))))